In [1]:
from config import gapi_key
import csv
import pandas as pd
import os
import requests
import gmaps

In [2]:
# Read in data from previous analysis
path = os.path.join('.', 'weather.csv')

weather = pd.read_csv(path)

weather.head()

,Latitude,Longitude,City,Country,City Latitude,City Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,-39.795553,-19.552937,jamestown,sh,-15.928634,-5.715175,73.89,73.0,58.0,10.18
1,-45.067772,34.285471,east london,za,-33.029158,27.854587,68.25,78.0,61.0,13.00
2,-61.761127,153.425467,hobart,au,-42.882138,147.327195,61.52,51.0,75.0,10.29
3,-30.664775,-60.061016,santa fe,ar,-31.610658,-60.697294,84.02,48.0,0.0,4.00
4,-65.803764,-74.911011,ushuaia,ar,-54.801912,-68.302951,50.00,71.0,75.0,11.41


## Create a Heat Map
Create a heat map that displays the humidity for every city from the part I of the homework.

In [3]:
gmaps.configure(api_key=gapi_key)

In [8]:
locations = weather[["City Latitude", "City Longitude"]].astype(float)
humidity = weather["Humidity (%)"]

In [12]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Find ideal places to vacation

In [13]:
# Narrow down the DataFrame to find ideal weather conditions

temp = weather['Temperature (F)']
humid = weather['Humidity (%)']
clouds = weather['Cloudiness (%)']
wind = weather['Wind Speed (mph)']

ideal = weather[(temp <= 90) & (temp >= 70) & (humid <= 50) & (clouds <= 25) & (wind <= 15)]
ideal

,Latitude,Longitude,City,Country,City Latitude,City Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
3,-30.664775,-60.061016,santa fe,ar,-31.610658,-60.697294,84.02,48.0,0.0,4.00
182,10.766946,-88.125789,corinto,ni,12.490741,-87.178433,87.80,48.0,20.0,4.70
329,12.091814,-102.824314,san jeronimo,mx,31.713671,-106.413351,75.54,40.0,0.0,1.99
459,24.314128,-94.495701,matamoros,mx,25.869029,-97.502738,71.62,25.0,5.0,5.21
510,-37.220289,-63.258913,santa rosa,ar,-36.620922,-64.291237,87.53,26.0,1.0,6.17


In [14]:
import requests
import json

### Get hotel

In [24]:

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# For each row in ideal df, find the first hotel that appears in Google Places search

hotels_info = []

for index, row in ideal.iterrows():
    
    h_info = {}
    
    lat = ideal.loc[index, 'City Latitude']
    lng = ideal.loc[index, 'City Longitude']
    city = ideal.loc[index, 'City'].title()
    country = ideal.loc[index, 'Country'].upper()
    coordinates = f"{lat}, {lng}"

    # set up search variables
    target_coordinates = coordinates
    target_radius = 5000
    target_type = "lodging"

    # declare the variables in the parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gapi_key,
    } 

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run the request
    response = requests.get(base_url, params=params)
    r = response.json()
    try:
        # print(r['results'][0])
        name = r['results'][0]['name']
        h_info = {
            'name': name,
            "city": city,
            "country": country,
            'location': (lat, lng) 
        }
        hotels_info.append(h_info)
    except:       
        print(coordinates)
        continue

print(hotels_info)

[{'name': 'Conquistador Hotel', 'city': 'Santa Fe', 'country': 'AR', 'location': (-31.6106578, -60.69729399999999)}, {'name': 'Hostal García', 'city': 'Corinto', 'country': 'NI', 'location': (12.490741400000001, -87.1784334)}, {'name': 'Hotel Mesaluna', 'city': 'San Jeronimo', 'country': 'MX', 'location': (31.713671, -106.4133507)}, {'name': 'Hotel Alameda Express', 'city': 'Matamoros', 'country': 'MX', 'location': (25.8690294, -97.5027376)}, {'name': 'Hotel Calfucurá', 'city': 'Santa Rosa', 'country': 'AR', 'location': (-36.620922, -64.2912369)}]


In [25]:
hotel_locations = [hotel['location'] for hotel in hotels_info]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
pop_ups = [info_box_template.format(**hotel) for hotel in hotels_info]

marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=pop_ups)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))